# Final Project - Machine Learning
## Submitted by: Muhammad Jawad Bashir
## Registration No. 330295 

In [307]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [308]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score
import sklearn.externals
import joblib
import time
from numpy import array
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_blobs
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

%matplotlib notebook

My Dataset is 11_ionsphere

In [309]:
dataset_ionsphere = '/content/drive/MyDrive/Datasets/11_ionosphere.csv'

In [310]:
df=pd.read_csv(dataset_ionsphere, delimiter=';')

In [311]:
df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,att17,att18,att19,att20,att21,att22,att23,att24,att25,att26,att27,att28,att29,att30,att31,att32,att33,label
0,1.0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1
1,1.0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0
2,1.0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1
3,1.0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0
4,1.0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1


In [312]:
print("Ionsphere set dimensions : {}".format(df.shape))

Ionsphere set dimensions : (351, 34)


In [313]:
df.groupby('label').size()

label
0    126
1    225
dtype: int64

In [314]:
data_descript = df.describe()
data_descript.iloc[:, :7]       

,att1,att2,att3,att4,att5,att6,att7
count,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000,351.000000
mean,0.783476,0.641342,0.044372,0.601068,0.115889,0.550095,0.119360
std,0.622309,0.497708,0.441435,0.519862,0.460810,0.492654,0.520750
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,1.000000,0.472135,-0.064735,0.412660,-0.024795,0.211310,-0.054840
50%,1.000000,0.871110,0.016310,0.809200,0.022800,0.728730,0.014710
75%,1.000000,1.000000,0.194185,1.000000,0.334655,0.969240,0.445675
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


###Generator Functions

In [315]:
import random
import sys
import os
import pandas as pd
import numpy as np
import math


# random feature generator
def random_features(df):
    df_rand =df.copy(deep=True)
    df_rand['r1'] =np.random.randint(-3, 3, df.shape[0])
    df_rand['r2'] =np.random.randint(-2, 2, df.shape[0])
    df_rand['r3'] =np.random.randint(-1, 1, df.shape[0])
    df_rand['r4'] =np.random.randint(0, 1, df.shape[0])
    df_rand['r5'] =np.random.randint(-1, 0, df.shape[0])
    return df_rand

# derived feature generator
def derived_features(df):
    df_derv =df.copy(deep=True)
    df_derv['d1'] =df['att2'].to_numpy() - df['att5'].to_numpy()
    df_derv['d2'] =df['att2'].to_numpy() + df['att3'].to_numpy()
    df_derv['d3'] =df['att20'].to_numpy() * df['att21'].to_numpy()
    df_derv['d4'] =df['att14'].to_numpy() * df['att13'].to_numpy()
    df_derv['d5'] =df['att19'].to_numpy()**2
    return df_derv

# PCA feature generator
def pca_features(df):
    y=df[['label']].to_numpy()
    x=df.drop(['label'], axis=1)
    norm_data=(x - x.mean()) / x.std()
    x_pca = PCA(0.95)
    x_pca.fit(norm_data)
    print ("retained components = ", x_pca.n_components_)
    cols = ['pca_%i' % i for i in range(1, (x_pca.n_components_)+1)]
    pca_df = DataFrame(x_pca.transform(x), columns=cols, index=x.index)
    pca_df['label']=y
    return pca_df

##Making Sets of Data

### 1. Original Features

In [316]:
org = df
org.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,att17,att18,att19,att20,att21,att22,att23,att24,att25,att26,att27,att28,att29,att30,att31,att32,att33,label
0,1.0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1
1,1.0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0
2,1.0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1
3,1.0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0
4,1.0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1


In [317]:
X_org =org.drop(['label'], axis=1)
Y_org =org[['label']].to_numpy()

### 2. Original + Random Features

In [318]:
org_rand = random_features(df)
org_rand.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,att17,att18,att19,att20,att21,att22,att23,att24,att25,att26,att27,att28,att29,att30,att31,att32,att33,label,r1,r2,r3,r4,r5
0,1.0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1,-1,1,0,0,-1
1,1.0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0,-1,-1,0,0,-1
2,1.0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1,-2,1,0,0,-1
3,1.0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0,0,0,-1,0,-1
4,1.0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1,-2,-1,-1,0,-1


In [319]:
X_org_rand =org_rand.drop(['label'], axis=1)
Y_org_rand =org_rand[['label']].to_numpy()

### 3. Original + Derived Features

In [320]:
org_derv = derived_features(df)
org_derv.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,att17,att18,att19,att20,att21,att22,att23,att24,att25,att26,att27,att28,att29,att30,att31,att32,att33,label,d1,d2,d3,d4,d5
0,1.0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1,0.97233,0.93650,-0.169056,-0.272079,0.103632
1,1.0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0,1.36156,0.81171,0.059574,0.360281,1.000000
2,1.0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1,0.99515,0.96635,-0.194953,0.004515,0.007293
3,1.0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0,0.00000,0.54839,-0.699750,-0.000000,0.296665
4,1.0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1,0.93469,0.97599,-0.021281,-0.002449,0.031730


In [321]:
X_org_derv =org_derv.drop(['label'], axis=1)
Y_org_derv =org_derv[['label']].to_numpy()

### 4. PCA

In [322]:
pca_only = pca_features(df)
pca_only.head()

retained components =  23


,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,pca_21,pca_22,pca_23,label
0,-2.393179,-0.811225,1.136915,-0.833459,-0.002376,-0.182644,0.705598,-0.122681,-0.403638,-0.484614,0.208984,-0.278674,0.246919,0.330952,-0.077725,-0.272723,-0.128287,-0.143593,-0.285179,-0.271545,-0.036105,-0.165851,-0.369702,1
1,-0.876896,-1.067564,0.137135,-0.693570,-0.233778,-0.382590,1.110257,-0.171680,-0.309206,-1.734726,-0.259118,-0.341181,0.204790,0.190596,-0.479570,-0.395637,-0.437476,-0.129618,-0.383349,-0.665669,-0.132648,0.187533,0.623618,0
2,-2.621249,-0.157211,1.522599,-0.512743,0.055090,0.045696,0.438598,-0.117803,-0.200194,-0.220707,0.224148,-0.255032,0.109872,0.011771,0.023138,-0.192307,-0.025321,-0.103413,-0.181336,-0.350322,0.074215,-0.226711,-0.240437,1
3,-0.639409,0.872907,0.746147,1.424902,-0.883423,-0.721661,0.270468,0.309988,0.468774,-1.682798,-0.988848,-0.222446,0.901003,0.645238,0.244895,0.805854,0.629538,-1.220425,0.165032,-0.326720,-0.914940,0.764227,-0.287486,0
4,-1.418359,-0.701517,1.267500,-1.362509,-0.167709,-0.100102,0.634593,0.143158,-0.577600,-0.673694,0.172297,-0.105260,0.008240,0.167199,-0.031786,-0.375207,0.035924,-0.323017,-0.310259,-0.361863,0.114653,-0.079546,-0.193811,1


In [323]:
X_pca =pca_only.drop(['label'], axis=1)
Y_pca =pca_only[['label']].to_numpy()

### 5. Original + Random Features + Derived + PCA

In [324]:
rand_features = org_rand.iloc[:,34:39]
rand_features.head()
derv_features = derv_rand.iloc[:,34:39]
derv_features.head()
pca_features = pca_only.drop(['label'], axis=1)
all_combined = pd.concat([org, rand_features, derv_features, pca_features], axis=1, join='inner')
all_combined.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,att14,att15,att16,att17,att18,att19,att20,att21,att22,att23,att24,att25,att26,att27,att28,att29,att30,att31,att32,att33,label,r1,r2,r3,r4,r5,d1,d2,d3,d4,d5,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,pca_10,pca_11,pca_12,pca_13,pca_14,pca_15,pca_16,pca_17,pca_18,pca_19,pca_20,pca_21,pca_22,pca_23
0,1.0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,1,-1,1,0,0,-1,0.97233,0.93650,-0.169056,-0.272079,0.103632,-2.393179,-0.811225,1.136915,-0.833459,-0.002376,-0.182644,0.705598,-0.122681,-0.403638,-0.484614,0.208984,-0.278674,0.246919,0.330952,-0.077725,-0.272723,-0.128287,-0.143593,-0.285179,-0.271545,-0.036105,-0.165851,-0.369702
1,1.0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,0.50874,-0.67743,0.34432,-0.69707,-0.51685,-0.97515,0.05499,-0.62237,0.33109,-1.00000,-0.13151,-0.45300,-0.18056,-0.35734,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,0,-1,-1,0,0,-1,1.36156,0.81171,0.059574,0.360281,1.000000,-0.876896,-1.067564,0.137135,-0.693570,-0.233778,-0.382590,1.110257,-0.171680,-0.309206,-1.734726,-0.259118,-0.341181,0.204790,0.190596,-0.479570,-0.395637,-0.437476,-0.129618,-0.383349,-0.665669,-0.132648,0.187533,0.623618
2,1.0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,0.73082,0.05346,0.85443,0.00827,0.54591,0.00299,0.83775,-0.13644,0.75535,-0.08540,0.70887,-0.27502,0.43385,-0.12062,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,1,-2,1,0,0,-1,0.99515,0.96635,-0.194953,0.004515,0.007293,-2.621249,-0.157211,1.522599,-0.512743,0.055090,0.045696,0.438598,-0.117803,-0.200194,-0.220707,0.224148,-0.255032,0.109872,0.011771,0.023138,-0.192307,-0.025321,-0.103413,-0.181336,-0.350322,0.074215,-0.226711,-0.240437
3,1.0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,-1.00000,0.14516,0.54094,-0.39330,-1.00000,-0.54467,-0.69975,1.00000,0.00000,0.00000,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,0,0,0,-1,0,-1,0.00000,0.54839,-0.699750,-0.000000,0.296665,-0.639409,0.872907,0.746147,1.424902,-0.883423,-0.721661,0.270468,0.309988,0.468774,-1.682798,-0.988848,-0.222446,0.901003,0.645238,0.244895,0.805854,0.629538,-1.220425,0.165032,-0.326720,-0.914940,0.764227,-0.287486
4,1.0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,0.52798,-0.20275,0.56409,-0.00712,0.34395,-0.27457,0.52940,-0.21780,0.45107,-0.17813,0.05982,-0.35575,0.02309,-0.52879,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,1,-2,-1,-1,0,-1,0.93469,0.97599,-0.021281,-0.002449,0.031730,-1.418359,-0.701517,1.267500,-1.362509,-0.167709,-0.100102,0.634593,0.143158,-0.577600,-0.673694,0.172297,-0.105260,0.008240,0.167199,-0.031786,-0.375207,0.035924,-0.323017,-0.310259,-0.361863,0.114653,-0.079546,-0.193811


In [325]:
X_all =all_combined.drop(['label'], axis=1)
Y_all =all_combined[['label']].to_numpy()

## Applying ML Algorithms

###K-Fold

In [326]:
#folds are 7
k=7
kf2=KFold(n_splits=k, shuffle=True)

#### A Function where we pass classifier and datasets and it returns Average Accuracy, Training time and Inference time

In [327]:
def k_fold_trainer(clf, X, Y):
    acc_score_kfold = []
    train_time=[]
    inf_time=[]

    for train_ind,test_ind in kf2.split(X, Y.ravel()):
        #print("Train:",train_ind,'Test:',test_ind)
        t1 = time.time()
        X_train,X_test = X.iloc[train_ind,:], X.iloc[test_ind,:]
        y_train,y_test = Y[train_ind], Y[test_ind]
        tt = time.time() - t1
        train_time.append(tt)

        t2 = time.time()
        clf.fit(X_train,y_train.ravel())
        pred_values = clf.predict(X_test)
        it=time.time()-t2
        inf_time.append(it)


        acc = accuracy_score(pred_values , y_test.ravel())
        acc_score_kfold.append(acc)
        
    avg_acc_score = sum(acc_score_kfold)/k
    training_time = sum(train_time)/k
    infer_time = sum(inf_time)/k
    return avg_acc_score, training_time, infer_time

##1. SVC



In [328]:
from sklearn.svm import SVC
clfSVC = SVC(kernel='rbf', C=10)

In [329]:
print("*******Results on SVC Classifier*******")
print("\nOriginal")
a, t, i = k_fold_trainer(clfSVC, X_org, Y_org)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOriginal + Random")
a, t, i = k_fold_trainer(clfSVC, X_org_rand, Y_org_rand)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOriginal + Derived")
a, t, i = k_fold_trainer(clfSVC, X_org_derv, Y_org_derv)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nPCA Features")
a, t, i = k_fold_trainer(clfSVC, X_pca, Y_pca)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOrigina + Random + Derived + PCA")
a, t, i = k_fold_trainer(clfSVC, X_all, Y_all)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

*******Results on SVC Classifier*******

Original
Average 7-fold Accuracy: 0.94593837535014
 Training Time: 0.0006946155003138951
 Inference Time: 0.007094008581978934 
 

Original + Random
Average 7-fold Accuracy: 0.9231932773109245
 Training Time: 0.0006453309740339007
 Inference Time: 0.007518972669328962 
 

Original + Derived
Average 7-fold Accuracy: 0.9516526610644257
 Training Time: 0.0006059237888881139
 Inference Time: 0.007089785167149135 
 

PCA Features
Average 7-fold Accuracy: 0.9430252100840335
 Training Time: 0.0008317743028913226
 Inference Time: 0.007955585207257952 
 

Origina + Random + Derived + PCA
Average 7-fold Accuracy: 0.9317086834733893
 Training Time: 0.0007145404815673828
 Inference Time: 0.007914611271449499 
 


##2. Decision Tree

In [330]:
from sklearn.tree import DecisionTreeClassifier
clfDT = DecisionTreeClassifier(max_depth=5)

In [331]:
print("*******Results on Decision Tree Classifier*******")
print("\nOriginal")
a, t, i = k_fold_trainer(clfDT, X_org, Y_org)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOriginal + Random")
a, t, i = k_fold_trainer(clfDT, X_org_rand, Y_org_rand)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOriginal + Derived")
a, t, i = k_fold_trainer(clfDT, X_org_derv, Y_org_derv)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nPCA Features")
a, t, i = k_fold_trainer(clfDT, X_pca, Y_pca)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOrigina + Random + Derived + PCA")
a, t, i = k_fold_trainer(clfDT, X_all, Y_all)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

*******Results on Decision Tree Classifier*******

Original
Average 7-fold Accuracy: 0.868795518207283
 Training Time: 0.0005901541028703962
 Inference Time: 0.007683719907488141 
 

Original + Random
Average 7-fold Accuracy: 0.8803921568627452
 Training Time: 0.0007391997746058873
 Inference Time: 0.009219510214669364 
 

Original + Derived
Average 7-fold Accuracy: 0.9116526610644257
 Training Time: 0.0005598749433244978
 Inference Time: 0.008153029850551061 
 

PCA Features
Average 7-fold Accuracy: 0.8859943977591037
 Training Time: 0.0006186962127685547
 Inference Time: 0.007357222693307059 
 

Origina + Random + Derived + PCA
Average 7-fold Accuracy: 0.8972549019607844
 Training Time: 0.0008633136749267578
 Inference Time: 0.01437681061880929 
 


##3. Random Forest

In [332]:
from sklearn.ensemble import RandomForestClassifier
clfRF = RandomForestClassifier(n_estimators=10)

In [333]:
print("*******Results on Random Forest Classifier*******")
print("\nOriginal")
a, t, i = k_fold_trainer(clfRF, X_org, Y_org)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOriginal + Random")
a, t, i = k_fold_trainer(clfRF, X_org_rand, Y_org_rand)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOriginal + Derived")
a, t, i = k_fold_trainer(clfRF, X_org_derv, Y_org_derv)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nPCA Features")
a, t, i = k_fold_trainer(clfRF, X_pca, Y_pca)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOrigina + Random + Derived + PCA")
a, t, i = k_fold_trainer(clfRF, X_all, Y_all)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

*******Results on Random Forest Classifier*******

Original
Average 7-fold Accuracy: 0.9373109243697478
 Training Time: 0.0007086140768868583
 Inference Time: 0.02776380947657994 
 

Original + Random
Average 7-fold Accuracy: 0.9259943977591037
 Training Time: 0.0007960796356201172
 Inference Time: 0.02589062282017299 
 

Original + Derived
Average 7-fold Accuracy: 0.9258823529411765
 Training Time: 0.0008401530129568917
 Inference Time: 0.028557436806815013 
 

PCA Features
Average 7-fold Accuracy: 0.9346778711484595
 Training Time: 0.0006036077226911272
 Inference Time: 0.024068049022129605 
 

Origina + Random + Derived + PCA
Average 7-fold Accuracy: 0.9372549019607843
 Training Time: 0.0008882113865443639
 Inference Time: 0.030548606600080217 
 


##4. Logistic Regression

In [334]:
from sklearn.linear_model import LogisticRegression
clfLR = LogisticRegression()

In [335]:
print("*******Results on Logistic Regression Classifier*******")
print("\nOriginal")
a, t, i = k_fold_trainer(clfLR, X_org, Y_org)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOriginal + Random")
a, t, i = k_fold_trainer(clfLR, X_org_rand, Y_org_rand)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOriginal + Derived")
a, t, i = k_fold_trainer(clfLR, X_org_derv, Y_org_derv)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nPCA Features")
a, t, i = k_fold_trainer(clfLR, X_pca, Y_pca)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

print("\nOrigina + Random + Derived + PCA")
a, t, i = k_fold_trainer(clfLR, X_all, Y_all)
print ('Average 7-fold Accuracy: {}\n Training Time: {}\n Inference Time: {} \n '.format(a,t,i))

*******Results on Logistic Regression Classifier*******

Original
Average 7-fold Accuracy: 0.8805042016806723
 Training Time: 0.0009263924189976283
 Inference Time: 0.022726467677525113 
 

Original + Random
Average 7-fold Accuracy: 0.8803921568627452
 Training Time: 0.0010198525020054408
 Inference Time: 0.025053024291992188 
 

Original + Derived
Average 7-fold Accuracy: 0.9003361344537815
 Training Time: 0.0008427756173270089
 Inference Time: 0.019093070711408342 
 

PCA Features
Average 7-fold Accuracy: 0.8805042016806723
 Training Time: 0.0008343628474644252
 Inference Time: 0.01460991586957659 
 

Origina + Random + Derived + PCA
Average 7-fold Accuracy: 0.9002240896358543
 Training Time: 0.001041548592703683
 Inference Time: 0.027327265058244978 
 
